In [7]:
import tensorflow
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import cv2
import os
import cvlib as cv

In [2]:
# Define the path to your test image here
image_path = "Test_Images/image1.jpg"

# Read input image
image = cv2.imread(image_path)

if image is None:
    print("Could not read input image")
    exit()

In [3]:
# Load pre-trained model
model = load_model("my_model.h5")

In [4]:
# Detect faces in the image
face, confidence = cv.detect_face(image)

classes = ['man', 'woman']

In [5]:
# Loop through detected faces
for idx, f in enumerate(face):

    # Get corner points of face rectangle       
    (startX, startY) = f[0], f[1]
    (endX, endY) = f[2], f[3]

    # Draw rectangle over face
    cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)

    # Crop the detected face region
    face_crop = np.copy(image[startY:endY, startX:endX])

    # Preprocessing for gender detection model
    face_crop = cv2.resize(face_crop, (96, 96))
    face_crop = face_crop.astype("float") / 255.0
    face_crop = img_to_array(face_crop)
    face_crop = np.expand_dims(face_crop, axis=0)

    # Apply gender detection on face
    conf = model.predict(face_crop)[0]
    print(conf)
    print(classes)

    # Get label with max accuracy
    idx = np.argmax(conf)
    label = classes[idx]

    label = "{}: {:.2f}%".format(label, conf[idx] * 100)

    Y = startY - 10 if startY - 10 > 10 else startY + 10

    # Write label and confidence above face rectangle
    cv2.putText(image, label, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX,
                0.7, (0, 255, 0), 2)

1/1 [==============================] - 0s 163ms/step
[1.000000e+00 3.784729e-08]
['man', 'woman']
1/1 [==============================] - 0s 23ms/step
[7.385596e-05 9.999261e-01]
['man', 'woman']


In [6]:
# Display output
cv2.imshow("gender detection", image)

# Press any key to close window           
cv2.waitKey()

# Save output
cv2.imwrite("gender_detection.jpg", image)

# Release resources
cv2.destroyAllWindows()